



Building a pipeline for the entire ML Workflow
1. Feature Extraction
2. Building a neural network with keras
3. Training the data
4. Getting the necessary prediction.

In [ ]:
import os
# mathematical computation
import numpy as np
# manipulate csv
import pandas as pd
# Visualization
import seaborn as sns
import matplotlib.pyplot as plt
# Estimating stats model
import statsmodels.api as sm

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
# recursive feature elimination for 
from sklearn.feature_selection import RFE

%matplotlib inline

In [ ]:
fd_restaurants = pd.read_excel("../input/food-environment-atlas/DataDownload.xls",sheet_name="RESTAURANTS")

fd_restaurants=fd_restaurants[['FIPS','FFR14','PCH_FFR_09_14','FSR14','PCH_FSR_09_14','PC_FFRSALES12','PC_FSRSALES12']]
fd_priceTax = pd.read_excel("../input/food-environment-atlas/DataDownload.xls",sheet_name="PRICES_TAXES")
fd_priceTax = fd_priceTax[['FIPS','FOOD_TAX14','CHIPSTAX_STORES14','SODATAX_STORES14']]
fd_local=pd.read_excel("../input/food-environment-atlas/DataDownload.xls",sheet_name="LOCAL")
fd_local = fd_local[['FIPS','GHVEG_SQFT12','DIRSALES_FARMS12','DIRSALES12',
                    'VEG_FARMS12','FRESHVEG_FARMS12','ORCHARD_FARMS12','BERRY_FARMS12','SLHOUSE12']]
fd1=pd.merge(fd_restaurants,
                 fd_priceTax,
                 on='FIPS')
food_data=pd.merge(fd1,
                 fd_local,
                 on='FIPS')

In [ ]:
tax_data = pd.read_csv('../input/individual-income-tax-statistics/15zpallagi.csv')
tax_data = tax_data[['zipcode','A85770','A09400','A07180','A02650','NUMDEP','A18425','A03270','A02500',
                    'A19300','A03300','A10600','STATE','A18500','A00200']]

zip_data=pd.read_csv('../input/zipcodes-county-fips-crosswalk/ZIP-COUNTY-FIPS_2017-06.csv')
zip_data=zip_data.rename(columns={'ZIP':'zipcode','STCOUNTYFP':'FIPS'})
print(tax_data.head())

In [ ]:
#MERGING THREE DATASETS
result = pd.merge(tax_data,
                 zip_data[['zipcode', 'STATE', 'FIPS']],
                 on='zipcode')

tax_data=pd.merge(result,food_data,on='FIPS')

print(tax_data.head())
print(tax_data.info())



In [ ]:
tax_data=tax_data.drop("STATE_y",1) 
tax_data=tax_data.drop("FIPS",1) 
tax_data=tax_data.drop("zipcode",1) 
tax_data.to_csv("tax_data.csv")

print(list(tax_data))

#One hot encoding of Y-values
y=tax_data['STATE_x']
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(y)
print(integer_encoded)
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
print(onehot_encoded)
tax_data=tax_data.drop("STATE_x",1) 



In [ ]:
tax_data.describe()

## Filter method

In [ ]:
field_def=pd.read_csv('../input/individual-income-tax-statistics/field_definitions.csv')
field_def=field_def[['Variable_Name','Description']]
food_def = pd.read_excel("../input/food-environment-atlas/DataDownload.xls",sheet_name="Variable List")
food_def=food_def.rename(columns={'Variable Name':'Variable_Name','Variable Code':'Description'})
food_def=food_def[['Variable_Name','Description']]
test=pd.concat([field_def,food_def])
#field_def=field_def.concat(food_def)
print(test[['Variable_Name','Description']])
field_def.set_index('Variable_Name',inplace=True)
fields